# start

In [2]:
PRFX = 'MelEda0624A2'
DEBUG = False

SEED = 101
DEVICE = 'cpu'; PIN_MEM = (DEVICE=='cuda')
FP16 = True

In [5]:
ls ../input/andrewmvd--isic-2019/

ISIC_2019_Training_GroundTruth.csv  ISIC_2019_Training_Metadata.csv
ISIC_2019_Training_Input/           isic-2019.zip


# setup

In [6]:
from pathlib import Path
import pickle
import pandas as pd
import random
import os
import numpy as np
import datetime
from collections import defaultdict, Counter

def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

import torch
device=torch.device(DEVICE)
import torch.nn as nn
import torch.nn.functional as F

from efficientnet_pytorch import EfficientNet

from apex import amp

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [7]:
p_19 = '../input/andrewmvd--isic-2019'
# p_out=f'../output/{PRFX}'; Path(p_out).mkdir(exist_ok=True,parents=True)

# eda

In [9]:
ls $p_19

ISIC_2019_Training_GroundTruth.csv  ISIC_2019_Training_Metadata.csv
ISIC_2019_Training_Input/


In [22]:
train = pd.read_csv(f'{p_19}/ISIC_2019_Training_GroundTruth.csv')
train.shape

(25331, 10)

In [27]:
train19meta = pd.read_csv(f'{p_19}/ISIC_2019_Training_Metadata.csv')
train19meta.shape

(25331, 5)

In [23]:
train.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
train.describe()

,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
count,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.000000,25331.0
mean,0.178516,0.508270,0.131183,0.034227,0.103588,0.009435,0.009988,0.024792,0.0
std,0.382954,0.499941,0.337607,0.181815,0.304732,0.096677,0.099440,0.155493,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [18]:
train['target'] = train.MEL

In [19]:
train.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,target
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## target

In [20]:
train.MEL.mean()

0.1785164423038964

In [21]:
train.target.mean(), len(train), train.target.sum()

(0.1785164423038964, 25331, 4522.0)

# patient

In [30]:
train19meta.lesion_id.isna().sum()

2084

In [33]:
train19meta.lesion_id.value_counts().value_counts()

1     6788
2     2505
3     1306
4      468
5      248
6      198
7      112
8       59
9       54
10      24
12      19
11      18
13      10
14       7
19       6
20       6
17       5
18       3
15       3
16       3
26       1
27       1
21       1
24       1
31       1
Name: lesion_id, dtype: int64